<h2>Find coordinates of neighborhoods' centers and add 'hover' data</h2>

In [1]:
from geojson_utils import point_in_multipolygon
from geojson_utils import centroid
import pandas as pd
import json

In [2]:
data= pd.read_json('data/NeighborhoodTabulationAreas.geojson')

In [11]:
centers = pd.DataFrame(columns=['ntaname', 'centerLong', 'centerLat'])
for element in data['features']:
    box_str = str(element['geometry']['coordinates'][0])
    box_str = '{"type": "Polygon","coordinates": '+box_str+'}'
    box = json.loads(box_str)
    point = centroid(box)
    centers = centers.append({'ntaname': element['properties']['ntaname'].lower(), 'centerLong': point['coordinates'][0], 'centerLat': point['coordinates'][1]}, ignore_index=True)

In [12]:
centers

,ntaname,centerLong,centerLat
0,borough park,-73.988661,40.630950
1,murray hill,-73.809546,40.768352
2,east elmhurst,-73.868396,40.763352
3,hollis,-73.761137,40.710639
4,manhattanville,-73.953782,40.817976
...,...,...,...
190,ozone park,-73.847049,40.675599
191,lindenwood-howard beach,-73.858961,40.652852
192,university heights-morris heights,-73.915974,40.852522
193,new brighton-silver lake,-74.102861,40.630050


In [13]:
df = pd.read_csv('data/MergedTreesPropertiesData.csv')

In [14]:
df = pd.merge(df, centers, how='outer', on='ntaname')

In [15]:
df.drop(columns={'Unnamed: 0'}, inplace=True)
df.dropna(inplace=True)


In [16]:
df['hover']=''

In [17]:
for index, row in df.iterrows():
    new_hover=row['ntaname']+'<br>'+row['borough']+'<br>'+'trees/sq.mile: '+str(row['trees/sq.mile'])+'<br>'+'avg. landprice: '+ str(row['avg.landprice_thous$/acre']) + '<br>'+'trees health: '+str(row['health'])+'/15'
    df.at[index,'hover'] = new_hover


In [18]:
df.drop(columns={'borough_y', 'avtot'}, inplace = True)

In [19]:
df.hover[0]

'airport<br>queens<br>trees/sq.mile: 18.94<br>avg. landprice: 1995.03<br>trees health: 14.32/15'

In [20]:
df.to_csv('data\Trees_Properties_With_Centroids.csv')